In [2]:
import opensense_data_downloader_and_transformer as oddt

In [3]:
# Instead of this we download the full 4-year dataset now direclty with the browser from
# https://data.4tu.nl/datasets/be252844-b672-471e-8d69-27269a862ec1/1

#oddt.download_overeem_2019_large_CML_data_Netherlands(local_path="data")

In [1]:
ls -lh data

total 92501696
-rw-r--r--@ 1 chwala-c  901    35G Jun 15  2024 IDRawCMLdata.dat
-rw-r--r--@ 1 chwala-c  901   8.8G May 16 11:33 IDRawCMLdata.zip


In [2]:
!head data/IDRawCMLdata.dat

YStart XStart YEnd XEnd Frequency DateTime ES SES Pmin Pmax PathLength Vendor ID
52.21836 6.925286 52.229751 6.921736 37.912 201101122315 0 0 -50 -49 1.29 NOKIA 4066
52.051263 6.61872 52.040809 6.591671 39.2525 201101122315 0 0 -56 -54 2.185 NOKIA 4048
51.980836 6.706526 51.963466 6.708112 37.94 201101122315 39 0 -48 -47 1.934 NOKIA 4051
51.958728 6.68532 51.963466 6.708112 37.9925 201101122315 0 0 -58 -57 1.648 NOKIA 4052
51.992557 6.523821 52.011968 6.588369 39.179 201101122315 0 0 -52 -50 4.918 NOKIA 4046
51.932545 6.576183 51.941893 6.617466 39.2 201101122315 0 0 -63 -59 3.015 NOKIA 4047
52.220458 6.893139 52.217718 6.891229 37.9785 201101122315 26 0 -44 -43 0.331 NOKIA 4049
51.963466 6.708112 51.980836 6.706526 39.2 201101122315 48 0 -50 -49 1.934 NOKIA 4050
52.220047 6.903828 52.196864 6.86988 37.94 201101122315 0 0 -43 -42 3.464 NOKIA 4053


In [28]:
import pandas as pd

df = pd.read_csv(
    'data/IDRawCMLdata.dat',
    nrows=1000000,
    sep=" ",
    header=0,
)

df = df.rename(columns=
    {
        'YStart': 'site_0_lat',
        'XStart': 'site_0_lon',
        'YEnd': 'site_1_lat',
        'XEnd': 'site_1_lon',
        'Frequency': 'frequency',
        'DateTime': 'time',
        'ES': 'error_seconds',
        'SES': 'sever_error_seconds',
        'Pmin': 'pmin',
        'Pmax': 'pmax',
        'PathLength': 'length',
        'ID': 'cml_id', # this is not correct since these are sublinks, but I have no time to do it correctly...
     }
)
df

,site_0_lat,site_0_lon,site_1_lat,site_1_lon,frequency,time,error_seconds,sever_error_seconds,pmin,pmax,length,Vendor,cml_id
0,52.218360,6.925286,52.229751,6.921736,37.9120,201101122315,0,0,-50.0,-49.0,1.290,NOKIA,4066
1,52.051263,6.618720,52.040809,6.591671,39.2525,201101122315,0,0,-56.0,-54.0,2.185,NOKIA,4048
2,51.980836,6.706526,51.963466,6.708112,37.9400,201101122315,39,0,-48.0,-47.0,1.934,NOKIA,4051
3,51.958728,6.685320,51.963466,6.708112,37.9925,201101122315,0,0,-58.0,-57.0,1.648,NOKIA,4052
4,51.992557,6.523821,52.011968,6.588369,39.1790,201101122315,0,0,-52.0,-50.0,4.918,NOKIA,4046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,51.865358,4.223562,51.881089,4.263222,26.3830,201101150345,0,0,-49.4,-48.3,3.236,NEC,1026
999996,52.268097,5.581889,52.235425,5.474368,23.1700,201101150345,0,0,-48.4,-47.4,8.171,NEC,1027
999997,51.561011,5.454443,51.622732,5.506799,23.1700,201101150345,0,0,-50.0,-48.0,7.758,NEC,1028
999998,51.585367,4.812388,51.566676,4.873029,26.0820,201101150345,0,0,-50.3,-48.7,4.678,NEC,1029


Get CML metadata from first row per CML ID

In [39]:
g['site_0_lat']

3526      52.221059
7692      52.221059
12626     52.221059
17952     52.221059
23290     52.221059
            ...    
973758    52.221059
979103    52.221059
984441    52.221059
989767    52.221059
995090    52.221059
Name: site_0_lat, Length: 205, dtype: float64

In [57]:
metadata_dict = {}
for cml_id, g in  df.groupby('cml_id'):
    metadata_dict[f'{cml_id:04d}'] = {k: g[k].values[0] for k in [
        'site_0_lat',
        'site_0_lon',
        'site_1_lat',
        'site_1_lon',
        'frequency',
        'length',
    ]
    }
metadata_dict['0002']

{'site_0_lat': np.float64(51.499885),
 'site_0_lon': np.float64(4.055869),
 'site_1_lat': np.float64(51.510914),
 'site_1_lon': np.float64(3.884575),
 'frequency': np.float64(19.095),
 'length': np.float64(11.919)}

In [45]:
df

,site_0_lat,site_0_lon,site_1_lat,site_1_lon,frequency,time,error_seconds,sever_error_seconds,pmin,pmax,length,Vendor,cml_id
0,52.218360,6.925286,52.229751,6.921736,37.9120,201101122315,0,0,-50.0,-49.0,1.290,NOKIA,4066
1,52.051263,6.618720,52.040809,6.591671,39.2525,201101122315,0,0,-56.0,-54.0,2.185,NOKIA,4048
2,51.980836,6.706526,51.963466,6.708112,37.9400,201101122315,39,0,-48.0,-47.0,1.934,NOKIA,4051
3,51.958728,6.685320,51.963466,6.708112,37.9925,201101122315,0,0,-58.0,-57.0,1.648,NOKIA,4052
4,51.992557,6.523821,52.011968,6.588369,39.1790,201101122315,0,0,-52.0,-50.0,4.918,NOKIA,4046
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,51.865358,4.223562,51.881089,4.263222,26.3830,201101150345,0,0,-49.4,-48.3,3.236,NEC,1026
999996,52.268097,5.581889,52.235425,5.474368,23.1700,201101150345,0,0,-48.4,-47.4,8.171,NEC,1027
999997,51.561011,5.454443,51.622732,5.506799,23.1700,201101150345,0,0,-50.0,-48.0,7.758,NEC,1028
999998,51.585367,4.812388,51.566676,4.873029,26.0820,201101150345,0,0,-50.3,-48.7,4.678,NEC,1029


In [33]:
g.iloc[0, :]

site_0_lat                52.856064
site_0_lon                 5.889313
site_1_lat                52.822543
site_1_lon                 5.967407
frequency                     26.39
time                   201101140745
error_seconds                     0
sever_error_seconds               0
pmin                         -120.0
pmax                          -46.0
length                        6.435
Vendor                        NOKIA
cml_id                         5752
Name: 637064, dtype: object

In [14]:
pd.to_datetime(g.DateTime, format='%Y%m%d%H%M')

3526     2011-01-12 23:30:00
7692     2011-01-12 23:45:00
12626    2011-01-13 00:00:00
17952    2011-01-13 00:15:00
23290    2011-01-13 00:30:00
                 ...        
973758   2011-01-15 02:30:00
979103   2011-01-15 02:45:00
984441   2011-01-15 03:00:00
989767   2011-01-15 03:15:00
995090   2011-01-15 03:30:00
Name: DateTime, Length: 205, dtype: datetime64[ns]

In [18]:
dict(g.dtypes)

{'YStart': dtype('float64'),
 'XStart': dtype('float64'),
 'YEnd': dtype('float64'),
 'XEnd': dtype('float64'),
 'Frequency': dtype('float64'),
 'DateTime': dtype('int64'),
 'ES': dtype('int64'),
 'SES': dtype('int64'),
 'Pmin': dtype('float64'),
 'Pmax': dtype('float64'),
 'PathLength': dtype('float64'),
 'Vendor': dtype('O'),
 'ID': dtype('int64')}

## use dask.Dataframe

In [23]:
import dask.dataframe as dd

dtypes = {
    'YStart': 'float64',
    'XStart': 'float64',
    'YEnd': 'float64',
    'XEnd': 'float64',
    'Frequency': 'float64',
    'DateTime': 'int64',
    'ES': 'int64',
    'SES': 'int64',
    'Pmin': 'float64',
    'Pmax': 'float64',
    'PathLength': 'float64',
    'Vendor': 'O',
    'ID': 'int64'
}

df = dd.read_csv(
    'data/IDRawCMLdata.dat',
    sep=" ",
    header=0,
    blocksize='100MB',
    dtype=dtypes,
)
df

,YStart,XStart,YEnd,XEnd,Frequency,DateTime,ES,SES,Pmin,Pmax,PathLength,Vendor,ID
npartitions=378,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,string,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
